In [1]:
import torch
from torch.amp import autocast
import numpy as np
import pandas as pd
import psutil
import logging
import time
import random
from helical.models.hyena_dna import HyenaDNA, HyenaDNAConfig
from datasets import load_dataset
from tqdm import tqdm

2026-01-14 13:34:39,803 - WARNING:py.warnings:/opt/homebrew/Caskroom/miniforge/base/envs/.helical_conda_venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

2026-01-14 13:34:39,848 - INFO:datasets:PyTorch version 2.7.0 available.


## Task 1 : Profiling on Naive Inferencing

### Configurations to set before inferencing (for naive inferencing)

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # for dynamic switching between CPU and GPU based on availability 

MODEL_CONFIG = {
    "model_name": "hyenadna-tiny-1k-seqlen-d256", # Model name to use
    # "model_name": "hyenadna-tiny-1k-seqlen", # Alternate Model name to use
    "batch_size": 1, # Number of sequences to process in a batch
    "use_amp": False, # Use auto mixed precision for optimization
    "amp_dtype": torch.float16 # will use amp_dtype only when use_amp is set to True
}

DATA_CONFIG = {
    "sample_size": 50, # Change if we want a different sample size
    "number_of_perturbations": 1 # Change if we want more perturbations
}

In [3]:
# for benchmarking all the experiments
all_profiles = []

def load_hyena_model():
    """
    Loads the HyenaDNA model based on settings in config.py,
    moves it to the correct device, and returns the model object.
    """
    hyena_config = HyenaDNAConfig(
        model_name = MODEL_CONFIG["model_name"], # HyenaDNA models can be used here
    )
    model = HyenaDNA(configurer=hyena_config)

    # move model to use GPU if possible
    if DEVICE == "cuda":
        model.model.to(DEVICE)
        
    print("Model loaded successfully!")
    
    return model
  
def compare_embeddings(embedding1, embedding2, name1="Embedding 1", name2="Embedding 2"):
    """
    Compares two embedding tensors by calculating Mean Absolute Error and Cosine Similarity.

    Args:
        embedding1 (torch.Tensor): The first embedding tensor.
        embedding2 (torch.Tensor): The second embedding tensor.
        name1 (str): Name for the first embedding for printing.
        name2 (str): Name for the second embedding for printing.
    """
    # Ensure tensors are on the same device (CPU) for comparison and are float32
    emb1_cpu = embedding1.cpu().to(torch.float32)
    emb2_cpu = embedding2.cpu().to(torch.float32)

    # 1. Calculate Mean Absolute Error (MAE)
    mae = torch.mean(torch.abs(emb1_cpu - emb2_cpu))

    # 2. Calculate Cosine Similarity
    # We compute it for each pair of vectors (row-wise) and then average
    cos_sim = torch.nn.functional.cosine_similarity(emb1_cpu, emb2_cpu, dim=1)
    avg_cos_sim = torch.mean(cos_sim)

    print(f"--- Embedding Comparison: {name1} vs. {name2} ---")
    print(f"Mean Absolute Error:     {mae.item():.10f}")
    print(f"Average Cosine Similarity: {avg_cos_sim.item():.10f}")

  
def get_sequences(sample_size: int):
    """Download the promoter_tata dataset and returns a sample of sequences

    Args:
        sample_size (int): Number of samples we want to use
    """
    print("Downloading dataset ...")
    dataset = load_dataset(
        "InstaDeepAI/nucleotide_transformer_downstream_tasks",
        trust_remote_code=True
    ).filter(lambda x: x["task"] == "promoter_tata")
    
    sequences = dataset["train"]["sequence"]
    print(f"Dataset is loaded, we will be taking a sample of {sample_size}")
    return sequences[:sample_size]
  

def add_pertubations(sequence_string: str, num_of_pertubations: int):
  """adds pertubations to a sequence of nucleotides"""
  nucleotides = ["A", "G", "T", "C"]
  length = len(sequence_string)
  seq_list = list(sequence_string)

  for _ in range(num_of_pertubations):
    random_idx = np.random.randint(0, length - 1)

    original_nucleotide = seq_list[random_idx]
    possible_pertubations = [n for n in nucleotides if n != original_nucleotide]
    new_nucleotide = random.choice(possible_pertubations)

    # apply the pertubation to mutate
    seq_list[random_idx] = new_nucleotide

  # return perturbed sequence
  return "".join(seq_list)

def log_inference_profile(
  total_time: float,
  latencies: list,
  num_samples: int,
  start_rss_mb: float
):
  """
  Calculates and logs inference related metrics of the run
  """
  avg_latency = np.mean(latencies) * 1000 # to convert in ms
  throughput = DATA_CONFIG["sample_size"]/total_time
  end_rss_mb = psutil.Process().memory_info().rss / (1024 * 1024)
  
  # get GPU memory if running on GPU
  peak_gpu_mb = 0
  if DEVICE == "cuda" and torch.cuda.is_available():
    peak_gpu_mb = torch.cuda.max_memory_allocated() / (1024 * 1024)
    
  log_line = f"""
------------ Inference Profile ------------
Device:                 {DEVICE.upper()}
Model name:             {MODEL_CONFIG["model_name"]}
amp enabled:            {MODEL_CONFIG["use_amp"]}
amp dtype:              {MODEL_CONFIG["amp_dtype"] if MODEL_CONFIG["use_amp"] else "torch.float32 (default)"}
Total Samples:          {num_samples}
Batch Size:             {MODEL_CONFIG["batch_size"]}
---
Total Time:             {total_time:.2f} s
Throughput:             {throughput:.2f} samples/s
Avg. Latency / Batch:   {avg_latency:.2f} ms
---
CPU RAM Usage:          {end_rss_mb - start_rss_mb:.2f} MB
Peak GPU Memory:        {peak_gpu_mb:.2f} MB
---------------------------------------------------
"""
  # Create a dictionary of the results
  profile_results = {
      "Model name":  MODEL_CONFIG["model_name"],
      "Total samples": num_samples,
      "Total time": f"{total_time:.2f} s",
      "Throughput (samples/s)": f"{throughput:.2f}",
      "Avg. Latency (ms/batch)": f"{avg_latency:.2f}",
      "CPU RAM Usage (MB)": f"{end_rss_mb - start_rss_mb:.2f}",
      "Peak GPU Memory (MB)": f"{peak_gpu_mb:.2f}"
  }
  
  # Reset peak memory stats for the next run if needed
  if DEVICE == "cuda" and torch.cuda.is_available():
      torch.cuda.reset_peak_memory_stats()
      
  return log_line, profile_results


def run_hyena_inferencing(model, sequences_to_process: list):
    """runs inferencing on sequences"""
    pertubation_embeddings = []
    latencies = []

    # Inference on pertubations
    overall_start = time.time()
    start_rss = psutil.Process().memory_info().rss / (1024 * 1024)
    BATCH_SIZE = MODEL_CONFIG["batch_size"]
    overall_start = time.time()

    for i in range(0, DATA_CONFIG["sample_size"], BATCH_SIZE):
        t_loop_in = time.time()
        raw_tokens = model.process_data(sequences_to_process[i:i + BATCH_SIZE])
        input_ids_tensor = torch.tensor(raw_tokens["input_ids"]).to(DEVICE)

        with torch.no_grad():
            with autocast(DEVICE, enabled=MODEL_CONFIG["use_amp"], dtype=MODEL_CONFIG["amp_dtype"]):
                outputs = model.model(input_ids=input_ids_tensor)
                embeddings = outputs

        t_loop_out = time.time()
        latencies.append(t_loop_out - t_loop_in)
        
        if isinstance(embeddings, torch.Tensor):
            pertubation_embeddings.append(embeddings)

    total_time = time.time() - overall_start

    # Call the logging function from utils
    run_profile, run_profile_dict = log_inference_profile(
        total_time=total_time,
        latencies=latencies,
        num_samples=len(sequences_to_process),
        start_rss_mb=start_rss
    )
    
    print(run_profile)

    return torch.cat(pertubation_embeddings, dim=0), run_profile_dict

### Profiling the inferencing on perturbated sequences (NAIVE)

In [4]:
time_signature = time.strftime('%Y%m%d-%H%M%S')

sequences = get_sequences(DATA_CONFIG["sample_size"])

perturbed_sequences = []

for sequence in sequences:
    perturbed_sequences.append(add_pertubations(sequence, num_of_pertubations=1))

perturbed_sequences = [add_pertubations(seq, num_of_pertubations=DATA_CONFIG["number_of_perturbations"]) for seq in sequences]

print("Loading Hyena model...")
model = load_hyena_model()

print("Starting inference run on original sequences...")
naive_original_embeddings, naive_original_profile = run_hyena_inferencing(model, sequences)
naive_original_profile["Sequences"] = "Original"
naive_original_profile["Experiment"] = "Naive (BS=1, FP32)"
all_profiles.append(naive_original_profile)

print("Starting inference run on perturbed sequences...")
naive_perturbed_embeddings, naive_perturbed_profile = run_hyena_inferencing(model, perturbed_sequences)
naive_perturbed_profile["Sequences"] = "Perturbed"
naive_perturbed_profile["Experiment"] = "Naive (BS=1, FP32)"
all_profiles.append(naive_perturbed_profile)


print(f"Successfully generated original embeddings of shape: {naive_original_embeddings.shape}")
print(f"Successfully generated perturbed embeddings of shape: {naive_perturbed_embeddings.shape}")

original_embedding_array = np.stack(naive_original_embeddings)
perturbed_embedding_array = np.stack(naive_perturbed_embeddings)

np.save(f"original_embedding_{time_signature}.npy", original_embedding_array)
np.save(f"perturbed_embedding_{time_signature}.npy", original_embedding_array)

2026-01-14 13:34:44,108 - INFO:helical.models.hyena_dna.pretrained_model:Loaded pretrained weights ok!
2026-01-14 13:34:44,109 - INFO:helical.models.hyena_dna.model:Model finished initializing.
2026-01-14 13:34:44,110 - INFO:helical.models.hyena_dna.model:'hyenadna-tiny-1k-seqlen-d256' model is in 'eval' mode, on device 'cpu'.
2026-01-14 13:34:44,111 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:44,115 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:44,153 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:44,156 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.


Dataset is loaded, we will be taking a sample of 50
Loading Hyena model...
Model loaded successfully!
Starting inference run on original sequences...


2026-01-14 13:34:44,196 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:44,199 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:44,239 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:44,243 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:44,283 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:44,286 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:44,326 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:44,329 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:44,369 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:44,372 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:3


------------ Inference Profile ------------
Device:                 CPU
Model name:             hyenadna-tiny-1k-seqlen-d256
amp enabled:            False
amp dtype:              torch.float32 (default)
Total Samples:          50
Batch Size:             1
---
Total Time:             2.13 s
Throughput:             23.49 samples/s
Avg. Latency / Batch:   42.57 ms
---
CPU RAM Usage:          28.55 MB
Peak GPU Memory:        0.00 MB
---------------------------------------------------

Starting inference run on perturbed sequences...


2026-01-14 13:34:46,491 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:46,495 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:46,534 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:46,537 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:46,576 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:46,579 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:46,619 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:46,622 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:46,662 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:46,665 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:3


------------ Inference Profile ------------
Device:                 CPU
Model name:             hyenadna-tiny-1k-seqlen-d256
amp enabled:            False
amp dtype:              torch.float32 (default)
Total Samples:          50
Batch Size:             1
---
Total Time:             2.17 s
Throughput:             23.09 samples/s
Avg. Latency / Batch:   43.30 ms
---
CPU RAM Usage:          13.84 MB
Peak GPU Memory:        0.00 MB
---------------------------------------------------

Successfully generated original embeddings of shape: torch.Size([50, 302, 256])
Successfully generated perturbed embeddings of shape: torch.Size([50, 302, 256])


# Task 2: Scale ISP and Optimizations

### Optimization 1 : Batching (Batch Size = 32)

### Configurations to set before inferencing (for batching optimization)

In [5]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # for dynamic switching between CPU and GPU based on availability 

MODEL_CONFIG = {
    "model_name": "hyenadna-tiny-1k-seqlen-d256", # Model name to use
    # "model_name": "hyenadna-tiny-1k-seqlen", # Alternate Model name to use
    "batch_size": 32, # Number of sequences to process in a batch
    "use_amp": False, # Use auto mixed precision for optimization
    "amp_dtype": torch.float16 # will use amp_dtype only when use_amp is set to True
}

DATA_CONFIG = {
    "sample_size": 50, # Change if we want a different sample size
    "number_of_perturbations": 1 # Change if we want more perturbations
}

In [6]:
time_signature = time.strftime('%Y%m%d-%H%M%S')

sequences = get_sequences(DATA_CONFIG["sample_size"])

perturbed_sequences = []

for sequence in sequences:
    perturbed_sequences.append(add_pertubations(sequence, num_of_pertubations=1))

perturbed_sequences = [add_pertubations(seq, num_of_pertubations=DATA_CONFIG["number_of_perturbations"]) for seq in sequences]

print("Loading Hyena model...")
model = load_hyena_model()

print("Starting inference run on original sequences...")
batch_only_original_embeddings, optimization_one_original_profile = run_hyena_inferencing(model, sequences)
optimization_one_original_profile["Sequences"] = "Original"
optimization_one_original_profile["Experiment"] = "Batching Only (BS=32, FP32)"
all_profiles.append(optimization_one_original_profile)

print("Starting inference run on perturbed sequences...")
batch_only_perturbed_embeddings, optimization_one_perturbed_profile = run_hyena_inferencing(model, perturbed_sequences)
optimization_one_perturbed_profile["Sequences"] = "Perturbed"
optimization_one_perturbed_profile["Experiment"] = "Batching Only (BS=32, FP32)"
all_profiles.append(optimization_one_perturbed_profile)

print(f"Successfully generated original embeddings of shape: {batch_only_original_embeddings.shape}")
print(f"Successfully generated perturbed embeddings of shape: {batch_only_perturbed_embeddings.shape}")

original_embedding_array = np.stack(batch_only_original_embeddings)
perturbed_embedding_array = np.stack(batch_only_perturbed_embeddings)

np.save(f"original_embedding_{time_signature}.npy", original_embedding_array)
np.save(f"perturbed_embedding_{time_signature}.npy", original_embedding_array)

2026-01-14 13:34:49,805 - INFO:helical.models.hyena_dna.pretrained_model:Loaded pretrained weights ok!
2026-01-14 13:34:49,806 - INFO:helical.models.hyena_dna.model:Model finished initializing.
2026-01-14 13:34:49,806 - INFO:helical.models.hyena_dna.model:'hyenadna-tiny-1k-seqlen-d256' model is in 'eval' mode, on device 'cpu'.
2026-01-14 13:34:49,808 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:49,817 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.


Dataset is loaded, we will be taking a sample of 50
Loading Hyena model...
Model loaded successfully!
Starting inference run on original sequences...


2026-01-14 13:34:50,529 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:50,536 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:51,089 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:51,098 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.



------------ Inference Profile ------------
Device:                 CPU
Model name:             hyenadna-tiny-1k-seqlen-d256
amp enabled:            False
amp dtype:              torch.float32 (default)
Total Samples:          50
Batch Size:             32
---
Total Time:             1.28 s
Throughput:             39.08 samples/s
Avg. Latency / Batch:   639.76 ms
---
CPU RAM Usage:          261.16 MB
Peak GPU Memory:        0.00 MB
---------------------------------------------------

Starting inference run on perturbed sequences...


2026-01-14 13:34:51,771 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:51,778 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.



------------ Inference Profile ------------
Device:                 CPU
Model name:             hyenadna-tiny-1k-seqlen-d256
amp enabled:            False
amp dtype:              torch.float32 (default)
Total Samples:          50
Batch Size:             32
---
Total Time:             1.22 s
Throughput:             41.13 samples/s
Avg. Latency / Batch:   607.80 ms
---
CPU RAM Usage:          66.03 MB
Peak GPU Memory:        0.00 MB
---------------------------------------------------

Successfully generated original embeddings of shape: torch.Size([50, 302, 256])
Successfully generated perturbed embeddings of shape: torch.Size([50, 302, 256])


### Optimization 2 : Mixed Precision (Batch Size = 32)

### Configurations to set before inferencing (for batching optimization)

In [7]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu" # for dynamic switching between CPU and GPU based on availability 

MODEL_CONFIG = {
    "model_name": "hyenadna-tiny-1k-seqlen-d256", # Model name to use
    # "model_name": "hyenadna-tiny-1k-seqlen", # Alternate Model name to use
    "batch_size": 32, # Number of sequences to process in a batch
    "use_amp": True, # Use auto mixed precision for optimization
    "amp_dtype": torch.float16 # will use amp_dtype only when use_amp is set to True
}

DATA_CONFIG = {
    "sample_size": 50, # Change if we want a different sample size
    "number_of_perturbations": 1 # Change if we want more perturbations
}

In [8]:
time_signature = time.strftime('%Y%m%d-%H%M%S')

sequences = get_sequences(DATA_CONFIG["sample_size"])

perturbed_sequences = []

for sequence in sequences:
    perturbed_sequences.append(add_pertubations(sequence, num_of_pertubations=1))

perturbed_sequences = [add_pertubations(seq, num_of_pertubations=DATA_CONFIG["number_of_perturbations"]) for seq in sequences]

print("Loading Hyena model...")
model = load_hyena_model()

print("Starting inference run on original sequences...")
batch_and_amp_original_embeddings, optimization_two_original_profile = run_hyena_inferencing(model, sequences)
optimization_two_original_profile["Sequences"] = "Original"
optimization_two_original_profile["Experiment"] = "Batching with AMP (BS=32, FP16)"
all_profiles.append(optimization_two_original_profile)

print("Starting inference run on perturbed sequences...")
batch_and_amp_perturbed_embeddings, optimization_two_perturbed_profile = run_hyena_inferencing(model, perturbed_sequences)
optimization_two_perturbed_profile["Sequences"] = "Perturbed"
optimization_two_perturbed_profile["Experiment"] = "Batching with AMP (BS=32, FP16)"
all_profiles.append(optimization_two_perturbed_profile)

print(f"Successfully generated original embeddings of shape: {batch_and_amp_original_embeddings.shape}")
print(f"Successfully generated perturbed embeddings of shape: {batch_and_amp_perturbed_embeddings.shape}")

original_embedding_array = np.stack(batch_and_amp_original_embeddings)
perturbed_embedding_array = np.stack(batch_and_amp_perturbed_embeddings)

np.save(f"original_embedding_{time_signature}.npy", original_embedding_array)
np.save(f"perturbed_embedding_{time_signature}.npy", original_embedding_array)

2026-01-14 13:34:53,384 - INFO:helical.models.hyena_dna.pretrained_model:Loaded pretrained weights ok!
2026-01-14 13:34:53,385 - INFO:helical.models.hyena_dna.model:Model finished initializing.
2026-01-14 13:34:53,385 - INFO:helical.models.hyena_dna.model:'hyenadna-tiny-1k-seqlen-d256' model is in 'eval' mode, on device 'cpu'.
2026-01-14 13:34:53,387 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:53,396 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.


Dataset is loaded, we will be taking a sample of 50
Loading Hyena model...
Model loaded successfully!
Starting inference run on original sequences...


2026-01-14 13:34:53,984 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:53,991 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.
2026-01-14 13:34:54,333 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:54,342 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.



------------ Inference Profile ------------
Device:                 CPU
Model name:             hyenadna-tiny-1k-seqlen-d256
amp enabled:            True
amp dtype:              torch.float16
Total Samples:          50
Batch Size:             32
---
Total Time:             0.95 s
Throughput:             52.88 samples/s
Avg. Latency / Batch:   472.72 ms
---
CPU RAM Usage:          77.39 MB
Peak GPU Memory:        0.00 MB
---------------------------------------------------

Starting inference run on perturbed sequences...


2026-01-14 13:34:54,929 - INFO:helical.models.hyena_dna.model:Processing data for HyenaDNA.
2026-01-14 13:34:54,935 - INFO:helical.models.hyena_dna.model:Succesfully prepared the HyenaDNA Dataset.



------------ Inference Profile ------------
Device:                 CPU
Model name:             hyenadna-tiny-1k-seqlen-d256
amp enabled:            True
amp dtype:              torch.float16
Total Samples:          50
Batch Size:             32
---
Total Time:             0.94 s
Throughput:             53.13 samples/s
Avg. Latency / Batch:   470.55 ms
---
CPU RAM Usage:          8.61 MB
Peak GPU Memory:        0.00 MB
---------------------------------------------------

Successfully generated original embeddings of shape: torch.Size([50, 302, 256])
Successfully generated perturbed embeddings of shape: torch.Size([50, 302, 256])


# Summarizing Results

In [9]:
df = pd.DataFrame(all_profiles)
df

,Model name,Total samples,Total time,Throughput (samples/s),Avg. Latency (ms/batch),CPU RAM Usage (MB),Peak GPU Memory (MB),Sequences,Experiment
0,hyenadna-tiny-1k-seqlen-d256,50,2.13 s,23.49,42.57,28.55,0.00,Original,"Naive (BS=1, FP32)"
1,hyenadna-tiny-1k-seqlen-d256,50,2.17 s,23.09,43.30,13.84,0.00,Perturbed,"Naive (BS=1, FP32)"
2,hyenadna-tiny-1k-seqlen-d256,50,1.28 s,39.08,639.76,261.16,0.00,Original,"Batching Only (BS=32, FP32)"
3,hyenadna-tiny-1k-seqlen-d256,50,1.22 s,41.13,607.80,66.03,0.00,Perturbed,"Batching Only (BS=32, FP32)"
4,hyenadna-tiny-1k-seqlen-d256,50,0.95 s,52.88,472.72,77.39,0.00,Original,"Batching with AMP (BS=32, FP16)"
5,hyenadna-tiny-1k-seqlen-d256,50,0.94 s,53.13,470.55,8.61,0.00,Perturbed,"Batching with AMP (BS=32, FP16)"


We can see that BATCHING optimization has reduced the inference time by 1.8x on both original and perturbed data when compared with Naive Inferencing

Also, further applying auto mixed precision (amp) to use FP16 futher reduced the inference time by 1.3x 

Both the optimization combined gives inference time reduction of 2.3x

### Validation of generated embeddings

Comparing original embeddings generated using Naive inferencing with Batch only optimized inferencing

In [10]:
compare_embeddings(naive_original_embeddings, batch_only_original_embeddings, 
                   name1="Naive (BS=1)", name2="Batched (BS=32)")

--- Embedding Comparison: Naive (BS=1) vs. Batched (BS=32) ---
Mean Absolute Error:     0.0000002314
Average Cosine Similarity: 1.0000000000


Comparing original embeddings generated using Naive inferencing with Batch + amp optimized inferencing

In [11]:
compare_embeddings(naive_original_embeddings, batch_and_amp_original_embeddings, 
                   name1="Naive (BS=1)", name2="Batching with AMP (BS=32, FP16)")

--- Embedding Comparison: Naive (BS=1) vs. Batching with AMP (BS=32, FP16) ---
Mean Absolute Error:     0.0004380150
Average Cosine Similarity: 0.9999992847


Perturbed Embedding should be have more differences

In [12]:
compare_embeddings(naive_original_embeddings, naive_perturbed_embeddings, 
                   name1="Naive (BS=1)", name2="Batched (BS=32)")

--- Embedding Comparison: Naive (BS=1) vs. Batched (BS=32) ---
Mean Absolute Error:     0.0199481137
Average Cosine Similarity: 0.9938756824


This suggests that we are able to optimize inference time by multifold factor without close to no compromise on embedding generations